# Feature transformation with Amazon SageMaker Processing and SparkML

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Spark in a managed SageMaker environment to run our preprocessing workload. Then, we'll take our preprocessed dataset and train a regression model using XGBoost.

![](images/processing.jpg)


## Contents

1. [Objective](#Objective:-predict-the-age-of-an-Abalone-from-its-physical-measurement)
1. [Setup](#Setup)
1. [Using Amazon SageMaker Processing to execute a SparkML Job](#Using-Amazon-SageMaker-Processing-to-execute-a-SparkML-Job)
  1. [Downloading dataset and uploading to S3](#Downloading-dataset-and-uploading-to-S3)
  1. [Build a Spark container for running the preprocessing job](#Build-a-Spark-container-for-running-the-preprocessing-job)
  1. [Run the preprocessing job using Amazon SageMaker Processing](#Run-the-preprocessing-job-using-Amazon-SageMaker-Processing)
    1. [Inspect the preprocessed dataset](#Inspect-the-preprocessed-dataset)
1. [Train a regression model using the Amazon SageMaker XGBoost algorithm](#Train-a-regression-model-using-the-SageMaker-XGBoost-algorithm)
  1. [Retrieve the XGBoost algorithm image](#Retrieve-the-XGBoost-algorithm-image)
  1. [Set XGBoost model parameters and dataset details](#Set-XGBoost-model-parameters-and-dataset-details)
  1. [Train the XGBoost model](#Train-the-XGBoost-model)

## Setup

Add the following policies to your SageMaker Execution Role:  
* `EC2ContainerRegistry`
* Permissions: `List`, `Read`, `Write` 
* Repository:  `amazon-reviews-spark-processor`

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [ ]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Using Amazon SageMaker Processing to Run a SparkML Transformation

### Downloading dataset and uploading to Amazon Simple Storage Service (Amazon S3)


Get name of previous Scikit Processing Job to use to construct the input for our Spark ML job.

In [ ]:
%store -r scikit_processing_job_name

In [ ]:
!aws s3 ls s3://$bucket/$scikit_processing_job_name/output/

In [ ]:
balanced_train_data_input = 's3://{}/{}/output/raw-labeled-split-balanced-header-train/'.format(bucket, scikit_processing_job_name)
balanced_validation_data_input = 's3://{}/{}/output/raw-labeled-split-balanced-header-validation/'.format(bucket, scikit_processing_job_name)
balanced_test_data_input = 's3://{}/{}/output/raw-labeled-split-balanced-header-test/'.format(bucket, scikit_processing_job_name)

In [ ]:
!aws s3 ls $balanced_train_data_input

### Build a Spark container for running the preprocessing job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset preprocessing.

Build the example Spark container.

In [ ]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [ ]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [ ]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

Create ECR repository and push docker image

In [ ]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

In [ ]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

In [ ]:
!docker tag $docker_repo:$docker_tag $image_uri

In [ ]:
!docker push $image_uri

### Run the preprocessing job using Amazon SageMaker Processing

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for preprocessing in the job configuration.

Review the Spark preprocessing script.

In [ ]:
cat preprocess-spark.py

In [ ]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=20, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.24xlarge',
#                            instance_type='ml.r5.8xlarge',
#                            max_runtime_in_seconds=600,
                            env={'mode': 'python'})

## Setup Input Data

In [ ]:
!aws s3 ls $balanced_train_data_input

## Setup Output Data

In [ ]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# TODO:  Clean these up
#input_raw_prefix = 'sagemaker/spark-preprocess-reviews-demo/input/raw/reviews'
output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)
#model_prefix = 'sagemaker/spark-preprocess-reviews-demo/model'
processing_job_name = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

In [ ]:
balanced_train_data_tfidf_output = 's3://{}/{}/tfidf-labeled-split-balanced-noheader-train'.format(bucket, output_prefix)
balanced_validation_data_tfidf_output = 's3://{}/{}/tfidf-labeled-split-balanced-noheader-validation'.format(bucket, output_prefix)
balanced_test_data_tfidf_output = 's3://{}/{}/tfidf-labeled-split-balanced-noheader-test'.format(bucket, output_prefix)

print(balanced_train_data_tfidf_output)
print(balanced_validation_data_tfidf_output)
print(balanced_test_data_tfidf_output)

## Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [ ]:
from sagemaker.processing import ProcessingOutput

# NOTE:  THIS IS NOW TESTING THE VALIDATION DATA SET (much smaller)

processor.run(code='preprocess-spark.py',
              arguments=['s3_input_data', balanced_train_data_input,
                         's3_output_data', balanced_train_data_tfidf_output,
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='dummy-output',
                                        source='/opt/ml/processing/output')
              ],          
              logs=True,
              wait=False
)

In [ ]:
preprocessing_job_description = processor.jobs[-1].describe()
print(preprocessing_job_description)

In [ ]:
processing_job_name = preprocessing_job_description['ProcessingJobName']

from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, processing_job_name)))


In [ ]:
processing_job_status = preprocessing_job_description['ProcessingJobStatus']

if (processing_job_status in ['Completed', 'Stopped']):
    # TODO:  Do something interesting
    print('Complete')
else:
    print(processing_job_status)

## Please wait until the Processing Job Completes above

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
                                                                            sagemaker_session=sagemaker_session)
running_processor.describe()

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, processing_job_name, region)))

# TODO:  Run the transformation for both validation and test

#### Inspect the processed dataset
Take a look at a few rows of the transformed dataset to make sure the preprocessing was successful.

In [ ]:
!aws s3 ls --recursive $balanced_train_data_tfidf_output/

In [ ]:
!aws s3 ls --recursive $balanced_validation_data_tfidf_output/

In [ ]:
!aws s3 ls --recursive $balanced_test_data_tfidf_output/